# Lab: Environment Characterization Experiment (ECE)

## Goals:

In this lab, you will use signal processing to extract information from an observed (simulated) signal. This is an example of a typical experimental procedure:
* a theory is formulated that describes how environment parameters impact aspects of an observable signal
* an experiment is conducted to collect the signal
* the signal is processed to extract the parameters of interest

In this lab, you will:

* formulate a model that describes how an observed signal depends on environmental parameters of interest
* derive how the environmental parameters can be determined from signal parameters
* process the derived signal to determine signal parameters and, then, the environmental parameters

## This notebook is incomplete

In this notebook, there are multiple places for you to fill in either code or text.

You should do that directly in this notebook. 

Once you have completed all your work in this notebook, rerun the entire notebook using "Kernel > Restart and Run All" from the menubar. 

Fix any errors, then remove this cell (your notebook is now complete), and submit.

In [ ]:
## the usual imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# load Audio widget
from IPython.display import Audio

## The Experiment

You are a member of a scientific team that explores distant planets, their moons, and other large objects in space. Your role is to devise and conduct an experiment to estimate 
* the density of the atmosphere, and
* the acceleration of gravity $g$
on a remote world.

The idea for the experiment is to use sound waves to measure these quantities indirectly. Specifically, the sound from a device that emits a sinusoidal tone will be recorded as that device moves through the atmosphere near the surface of the planet that is being explored.

You are looking to exploit two properties of sound propagation from or to an object in motion:
1. the sound travels at a velocity $c$ that is dependent on the density $\rho$ of the air in the atmosphere. Specifically, it is known that
$$
c = \sqrt{\gamma \cdot \frac{p}{\rho}},
$$
  * here $\gamma$ is the *adiabatic* index; the value of this index is approximately $\gamma \approx 1.4$
  * and $p$ is the pressure, which can be measured independently using a barometer
  * We will concern ourselves only with measuring the speed of sound $c$ and leave the computation of the density to other team members.
2. the frequency of sound changes when the distance from the object that emits the sound to the observer changes; this is known as the Doppler effect. The experiment subject the emitter to a trajectory that depends on the acceleration of gravity $g$.

### Putting the sound emitting object in motion

The primary device in the experiment is a small object that emits a constant tone of frequency $f_0$.

Using a small catapult, this object is launched straight up (away from the surface of the planet) at an initial, known velocity of $v_0$.

The acceleration of gravity $g$ first decelerates our object, then stops it momentarily, before it accelerates the object back to the launch site.

While the object travels along this trajectory, a recording device records the sound from the device. The recording device is located at the launch/landing location. We denote that location by $d=0$.

## Modeling: what should we be observing?

We will see momentarily that the recorded frequency depends on the speed $c$ at which sounds propagates and the acceleration of gravity $g$.

We know that 
* the phase $\phi$ of a tone (frequency $f_0$ changes proportional to the delay $\tau$ that the signal experiences as the signal propagates from emitter to receiver:
$$
\phi = -2\pi f_0 \tau,
$$
* the delay $\tau$ of the signal is proportional to the distance $d$ between emitter and receiver:
$$
\tau = \frac{d}{c},
$$
* the distance $d$ between emitter and receiver is governed by the laws of motion, specifically
$$
d = v_0 \cdot t - \frac{g}{2} \cdot t^2.
$$

### The received signal

From the above equations, we see that the phase of the received signal
* varies with time
* is given by
$$
\phi(t) = -2\pi f_0 \cdot \frac{v_0}{c} \cdot t + 2\pi f_0 \cdot \frac{g}{2c} \cdot t^2.
$$

If the emitted signal is $s(t) = \cos(2\pi f_0 t + \phi_0)$, then the received signal equals
$$
r(t) = A(t) \cdot \cos(2\pi f_0 t + \phi(t) + \phi_0) = A(t) \cdot \cos(2\pi f_0 t  -2\pi f_0 \cdot \frac{v_0}{c} \cdot t + 2\pi f_0 \cdot \frac{g}{2c} \cdot t^2 + \phi_0),
$$
where $A(t)$ is the amplitude that varies with distance $d$ and, thus, with time. We will not need to model $A(t)$ as all relevant information is contained in the (instantaneous) frequency.

### Instantaneous frequency $f_i(t)$

The received signal $r(t)$ is not a simple sinusoid. The frequency of the received signal varies with time.

For a signal of the form $\cos(\Psi(t))$, the instantaneous frequency $f_i(t)$ is given by
$$
f_i(t) = \frac{1}{2\pi} \frac{d\Psi(t)}{dt}.
$$

For our received signal, $\Psi(t) = 2\pi f_0 t  -2\pi f_0 \cdot \frac{v_0}{c} \cdot t + 2\pi f_0 \cdot \frac{g}{2c} \cdot t^2 + \phi_0$.

Therefore, the instantaneous frequency of the received signal is:
$$
f_i(t) = f_0 - f_0 \cdot \frac{v_0}{c} + f_0 \cdot \frac{g}{c} \cdot t.
$$

The instantaneous frequency of the received signal:
* is initially lower than $f_0$, $f_i(0) = f_0 \cdot (1- \frac{v_0}{c})$
* then, the instantantaneous frequency increases linearly with slope $f_0 \cdot \frac{g}{c}$.

### Goal:

Thus, we want to measure the instantaneous frequency of the received signal and determine  
* the initial frequency $b=f_i(0)=f_0 \cdot (1- \frac{v_0}{c})$ and 
* the rate of change of the instantaneous frequency $m = f_0 \cdot \frac{g}{c}$. 

From these two measurements (and knowledge of $f_0$ and $v_0$) we can derive $c$ and $g$.

## The experiment: Obtaining the received signal

* The received signal `rr` is recorded and the code in the cell below loads this recording. 
 - The file `data.npz` must be in your working directory

* Additionally, the data file contains:
  - the transmitted signal `ss`
  - the sample rate `fs`
  - the frequency of the transmitted signals `f0`
  - the launch velocity `v0`

* After the code in the cell below is executed, these variables are available for you to work with.

In [ ]:
## load data from file
data_file = 'data.npz'

npz_file = np.load(data_file)

rr = npz_file['rr']   # received signal
ss = npz_file['ss']   # sent signal
fs= npz_file['fs'] * 1.    # sample rate
f0 = npz_file['f0'] * 1.   # frequency of emitted signal
v0 = npz_file['v0'] * 1.   # launch velocity

## Analysis of the received signal

We will analyze the received signal in a few steps:
1. by playing the signal through the speaker, we can form a qualitative impression of the signal
2. by plotting various segments of the signal, we can measure the frequency of the signal at different points in time
3. by computing and plotting the *spectrogram* of the signal, we can analyze the frequency content of the signal over time

The objective is to find:
* the initial frequency offset and
* the rate of change of the frequency
and from these two measurements, to determine environment parameters $g$ and $c$.

#### Task: Play the transmitted and received signal

Use the `Audio` widget that we loaded earlier to play both the transmitted and received signals.

Then describe *qualitatively* the differences between the two signals. Be as specific as you can about the chracteristics of the audible received signal.

In [ ]:
# Play the transmitted signal
Audio(ss, rate=fs)

In [ ]:
# Play the received signal
Audio(rr, rate=fs)

#### FILL THIS CELL

Use this cell to describe qualitatively the audible characteristics of the received signal

#### Task: Plot segments of the transmitted and received signals

After the qualitative assessment, we perform a more quantitative analysis of the signal. Our focus is on identifying the change in frequency of the received signal.

To that end, we examine the received (and transmitted) signals at three time instances during the experiment: 
* at the very beggining
* in the middle
* at the very end
Specifically, we will plot the signals at these instances.

Before we can make the plots, you need to construct a time grid.

In [ ]:
## Task: make a time grid using the sample rate fs and the length of the signals
# the length of tt must match the length of rr and ss
# the increment between successive elements of tt must eal 1/fs
tt = FILL_ME_IN

With the time grid in hand, we can now plot the signals.

The signals are too long to be plotted in their entirety - you would not be able to determine frequency.

Instead, we must examine short segments of the signal. Proceed as follows

* make a figure with three subplots
  - each figure shows the transmitted and received signals
* each subplot should show $N$ samples of both signals
  - find a value for $N$ that works well to measure frequency
* the first plot, shows the first $N$ samples (starts at time $t=0$)
* the second plot, shows $N$ samples in the very middle of the signal (centered at time $t=5$ s)
* the final plot shows the last $N$ samples (ends at time $t=10$ s)

Make sure that your time-axis shows the correct time for the signals you're plotting.

In [ ]:
## three-panel plot of the signals
N = FILL_ME_IN

plt.figure(layout='constrained')

plt.subplot(3, 1, 1)
FILL_ME_IN

plt.show()


### Task: Frequency Measurement

* The frequency of the received signal is different in each of the three subplots changes while the frequency of the emitted signal remains constant at $f_0$.

* In each of the three subplots, measure the frequency of the received signal and compile your measurements in the following table

|    Time $t$ (s)  |   Inst. Frequency $f_i(t)$ (Hz)   | 
|------------------|-----------------------------------|
|      0           |                                   |
|      5           |                                   |
|     10           |                                   |

### Task: Equation for $f_i(t)$

In the description of the experiment, we determined that the instantaneous frequency should change linearly with time $t$.

From the three frequency measurements that you compiled, determine the equation that describes how the instantaneous frequency $f_i(t)$ changes with time.
Specifically, find the offset $b$ and the slope $m$ of the linear equation
$$
f_i(t) = b + m \cdot t
$$
that fits your measurements.

Give your answers below; make sure to include proper units for $b$ and $m$.

* Measured initial frequency offset $b$ = FILL_ME_IN
* Measured slope of instantaneous frequency $m$ = FILL_ME_IN

### Spectrogram Analysis

A spectrogram is a tool for analyzing signal with time-varying frequency content. The frequency of the received signal is changing over time. Hence, the received signal is a good candidate for analysis via a spectrogram.

A spectrogram is a two-dimensional pseudo-color mesh plot. Its axes are time and frequency. The color coding indicates the magnitude of the spectrum at a given time.

Technically, the spectrogram is computed by 
* dividing the signal into short segments - similar to what you did with your plots above. 
* for each such segment, the magnitude of the spectrum is computed using the same procedure that we used last week.
* the results are collected into a two-dimensional array
  - one dimension corresponds to the frequency grid associated with the computation of the spectra
  - the other dimension corresponds to the time-instances when the samples for the segment were collected.
* this two-dimensinal array is plotted using `plt.pcolormesh()`

The spectrogram of the received signal is computed and plotted using `plt.specgram()`. 
* The parameter `NFFT` controls the length of the segments; it was chosen to give a frequency resolution of `fs/NFFT` of approximately 10Hz.
* The parameter `Fs` specifies the sample rate; it must be given to scale the axes properly
* Also, we zoom in on the frequencies that are of most interest to us.

**Task:** add a command to plot the line for the instantaneous frequency $f_i(t)$ on top of the spectrogram. Make the line red and dashed.
* if your computation of instantaneous frequency is correct, this line should follow the yellow high-light closely.

In [ ]:
## spectrogram
plt.specgram(rr, NFFT=2048, Fs=fs)

# Task: add a command to plot the line for the instantaneous frequency
FILL_ME_IN

plt.ylim(0, 2000)
plt.xlim(0, tt[-1])
plt.grid()
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.colorbar()

plt.show()

## Task: Determine the Environmant Parameters

Recall that analyzing the frequency contents was just a means to an end. Our real goal os to determine the two environment parameters
* acceleration of gravity $g$, and
* speed of sound propagation $c$

Use the measured initial frequency offset $b$ and the slope of the instantaneous frequency to determine the two parameters; make sure to include proper units.
* **Acceleration of gravity** $g$ = FILL_ME_IN
* **Speed of sound propagation** $c$ = FILL_ME_IN

## Summary

We simulated a simple experiment that allowed us to extract information from an observed signal. 

The theoretical basis for the experiment combined simple physics that allowed us to model and predict the outcome of the experiment. Specifically, the model predicted that frequency of the observed signal varies over time. It provided specific expressions that describe how this variation depends on the parameters of interest.

Armed with this theoretical basis, we:
- analyzed the received signal to measure teh variation in frequency
  - the instantaneous frequency ws measured manually in three different places
  - a spectrogram was used to confirm the manual measurement
- we determined the parameters of the instantaneous frequency
- from these signal parameters we determined the environment parameters of interest.